In [10]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
miniconda_path = r"C:\Users\lisa.smith\Miniconda3"
download_path = r"C:\Users\lisa.smith\Downloads"


## The Functions

In [11]:
def create_unique_IDs(user_initials):
    global batch_ID, start_date, timestamp
    user = user_initials+'-'
    currentDateAndTime = dt.now()
    curr_dt = dt.today().strftime('%Y-%m-%d-')
    currentTime = currentDateAndTime.strftime("%H-%M")
    timestamp = currentDateAndTime.timestamp()-1660000000
    start_date= dt.today().strftime('%Y-%m-%d %H:%M:%S')
    batch_ID = user + curr_dt + currentTime
    return  batch_ID,start_date,timestamp



In [12]:
create_unique_IDs('LS')
start_date

'2022-11-07 15:09:48'

## Part 1 Read File

In [13]:
%%time
AlteryxMB_path = r"S:\BSA SM Shared\11. Analytics & Reporting Team\11.20 The Move to Oracle\01.Process_Templates_and_data\17. How do I process Multiband"
filename = r"\multiband_reactivate.xlsx"
data=pd.read_excel(AlteryxMB_path + filename)
data.head(3)

CPU times: total: 2.2 s
Wall time: 2.81 s


,PG_ID,Oracle_name,NPC,CHARGE_LINE_NUMBER,ChargeLineIncrement,PRICING_UOM_CODE,Charge_ID,START_DATE,END_DATE,ITEM_CLASS_CODE,...,TIER3_MINcalc,TIER3_MAXcalc,TIER4_MINcalc,TIER4_MAXcalc,B1 EXVAT,AdjTL1,AdjTL2,AdjTL3,AdjTL4,Count_Dup
0,1,National,FCB18421,1,2,EA,2194,11-04-2022,06-09-2022,Category_Tower_02,...,19.0,34.0,34.0,NaN,32.000000,0.320,0.64,1.6,3.200,1
1,1,National,FCB18512,1,2,EA,2280,11-04-2022,06-09-2022,Category_Tower_02,...,19.0,34.0,34.0,NaN,29.990000,0.300,0.60,1.5,2.999,1
2,1,National,FAM20380,1,2,EA,300000125701696,10-05-2021,15-07-2022,Category_Tower_11,...,NaN,NaN,NaN,NaN,177.316667,1.767,3.55,NaN,NaN,1


## Remove Duplicate Pricelists and Create Unique ID

In [14]:
pricelists_lookup = data[['Oracle_name',]]
pricelist_unique=pricelists_lookup.drop_duplicates()
pricelist_unique['PRICELIST_ID'] = np.arange(len(pricelist_unique))+round(timestamp*100)
pricelist_unique.head(3)               

C:\Users\lisa.smith\AppData\Local\Temp\ipykernel_10132\3345299862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pricelist_unique['PRICELIST_ID'] = np.arange(len(pricelist_unique))+round(timestamp*100)


,Oracle_name,PRICELIST_ID
0,National,783378830
7,ICF NCF Band 3 50 74 9,783378831
77,Ncf Other Nhs Core Catalogue,783378832


## Create Unique IDs for Items and Charge and static parameters

In [16]:
# TODO - tidy first part of this code to be the same syntax as 2nd part
data['BATCH_NAME'] = batch_ID
data['SOURCE_PRICELIST_ITEM_ID'] = np.arange(len(MB_add_data))+round(timestamp*1000)
data['SOURCE_CHARGE_ID'] = np.arange(len(MB_add_data))+round(timestamp*10000)
data['OP_CODE_NO_OP'] = 'NO-OP'
data['OP_CODE_CREATE'] = 'CREATE'
data['START_DATE'] = start_date
data['SOURCE_TIERED_PRICING_ID']='TH_'+data['SOURCE_CHARGE_ID'].astype(str)

data = MB_add_data.assign(ITEM_LEVEL_CODE= 'ITEM', ITEM_ID= '',CH_PRICING_UOM='',PRIMARY_PRICING_UOM= 'Y', 
LINE_TYPE_CODE='ORA_BUY',PARENT_ENTITY_TYPE_CODE='PRICE_LIST_ITEM',
ChargeLine=MB_add_data['ChargeLineIncrement'], CHARGE_DEFINITION_CODE= 'QP_SALE_PRICE',CALCULATION_METHOD_CODE='PRICE',
NEW_SOURCE_CHARGE_ID='',PRICE_PERIODICITY_NAME='',PRICE_PERIODICITY='',CALCULATION_TYPE_CODE='',
COST_CALCULATION_AMOUNT='',CALCULATION_AMOUNT='',PRICING_BASIS='',PRICING_BASIS_ID='',CAN_ADJUST_FLAG='Y',
AGGREGATION_METHOD_CODE='ORA_ON_LINE',TIER_BASIS_TYPE_CODE='ORA_ITEM_QUANTITY',TIER_BASIS_NAME='',
TIER_BASIS_ID='',GRADUATED_CODE='HIGHEST_TIER',APPLICATION_METHOD_CODE='PER_UNIT',ADJUSTMENT_TYPE_CODE='DISCOUNT_AMOUNT',
ADJUSTMENT_BASIS_NAME='',ADJUSTMENT_BASIS_ID='',PARTIAL_BLOCK_ACTION_CODE='',ENFORCE_ADJ_CALCULATION_FLAG='Y',
ADJUSTMENT_BASIS_NAMEb='BASE_PRICE')
 
data.head(3)     


,PG_ID,Oracle_name,NPC,CHARGE_LINE_NUMBER,ChargeLineIncrement,PRICING_UOM_CODE,Charge_ID,START_DATE,END_DATE,ITEM_CLASS_CODE,...,TIER_BASIS_NAME,TIER_BASIS_ID,GRADUATED_CODE,APPLICATION_METHOD_CODE,ADJUSTMENT_TYPE_CODE,ADJUSTMENT_BASIS_NAME,ADJUSTMENT_BASIS_ID,PARTIAL_BLOCK_ACTION_CODE,ENFORCE_ADJ_CALCULATION_FLAG,ADJUSTMENT_BASIS_NAMEb
0,1,National,FCB18421,2,2,EA,2194,2022-11-07 15:09:48,06-09-2022,Category_Tower_02,...,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y,BASE_PRICE
1,1,National,FCB18512,2,2,EA,2280,2022-11-07 15:09:48,06-09-2022,Category_Tower_02,...,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y,BASE_PRICE
2,1,National,FAM20380,2,2,EA,300000125701696,2022-11-07 15:09:48,15-07-2022,Category_Tower_11,...,,,HIGHEST_TIER,PER_UNIT,DISCOUNT_AMOUNT,,,,Y,BASE_PRICE


## Join column with Pricelist IDs

In [17]:
data_joined = pd.merge(data,pricelist_unique,on='Oracle_name',how='inner')
data_joined.head(3)
print(np.shape(data_joined))

(2518, 62)


In [18]:
columns_list=list(data_joined.columns)
columns_list

['PG_ID',
 'Oracle_name',
 'NPC',
 'CHARGE_LINE_NUMBER',
 'ChargeLineIncrement',
 'PRICING_UOM_CODE',
 'Charge_ID',
 'START_DATE',
 'END_DATE',
 'ITEM_CLASS_CODE',
 'T1_var',
 'Num_tiers',
 'TIER1_MINcalc',
 'TIER1_MAXcalc',
 'TIER2_MINcalc',
 'TIER2_MAXcalc',
 'TIER3_MINcalc',
 'TIER3_MAXcalc',
 'TIER4_MINcalc',
 'TIER4_MAXcalc',
 'B1 EXVAT',
 'AdjTL1',
 'AdjTL2',
 'AdjTL3',
 'AdjTL4',
 'Count_Dup',
 'BATCH_NAME',
 'SOURCE_PRICELIST_ITEM_ID',
 'SOURCE_CHARGE_ID',
 'OP_CODE_NO_OP',
 'OP_CODE_CREATE',
 'SOURCE_TIERED_PRICING_ID',
 'ITEM_LEVEL_CODE',
 'ITEM_ID',
 'CH_PRICING_UOM',
 'PRIMARY_PRICING_UOM',
 'LINE_TYPE_CODE',
 'PARENT_ENTITY_TYPE_CODE',
 'CHARGE_DEFINITION_CODE',
 'CALCULATION_METHOD_CODE',
 'NEW_SOURCE_CHARGE_ID',
 'PRICE_PERIODICITY_NAME',
 'PRICE_PERIODICITY',
 'CALCULATION_TYPE_CODE',
 'COST_CALCULATION_AMOUNT',
 'CALCULATION_AMOUNT',
 'PRICING_BASIS',
 'PRICING_BASIS_ID',
 'CAN_ADJUST_FLAG',
 'AGGREGATION_METHOD_CODE',
 'TIER_BASIS_TYPE_CODE',
 'TIER_BASIS_NAME',
 'TIE

## Create CSV sheets

In [19]:
Pricelists_int = data_joined[['BATCH_NAME','OP_CODE_NO_OP','PRICELIST_ID','Oracle_name',]]
Pricelists_int_unique=Pricelists_int.drop_duplicates(subset=['Oracle_name'])
Pricelists_int_unique.head(3)

,BATCH_NAME,OP_CODE_NO_OP,PRICELIST_ID,Oracle_name
0,LS-2022-11-07-15-09,NO-OP,783378830,National
2496,LS-2022-11-07-15-09,NO-OP,783378831,ICF NCF Band 3 50 74 9
2500,LS-2022-11-07-15-09,NO-OP,783378832,Ncf Other Nhs Core Catalogue


In [20]:
PriceListItems_int=data_joined[['OP_CODE_NO_OP','PRICELIST_ID','SOURCE_PRICELIST_ITEM_ID','ITEM_LEVEL_CODE',
'NPC','ITEM_ID','CH_PRICING_UOM','PRICING_UOM_CODE','PRIMARY_PRICING_UOM','LINE_TYPE_CODE']]
PriceListItems_int

,OP_CODE_NO_OP,PRICELIST_ID,SOURCE_PRICELIST_ITEM_ID,ITEM_LEVEL_CODE,NPC,ITEM_ID,CH_PRICING_UOM,PRICING_UOM_CODE,PRIMARY_PRICING_UOM,LINE_TYPE_CODE
0,NO-OP,783378830,7833788297,ITEM,FCB18421,,,EA,Y,ORA_BUY
1,NO-OP,783378830,7833788298,ITEM,FCB18512,,,EA,Y,ORA_BUY
2,NO-OP,783378830,7833788299,ITEM,FAM20380,,,EA,Y,ORA_BUY
3,NO-OP,783378830,7833788300,ITEM,FCB19296,,,EA,Y,ORA_BUY
4,NO-OP,783378830,7833788301,ITEM,FAM20390,,,EA,Y,ORA_BUY
...,...,...,...,...,...,...,...,...,...,...
2513,NO-OP,783378838,7833790131,ITEM,FNG2247,,,PAC,Y,ORA_BUY
2514,NO-OP,783378839,7833790136,ITEM,FGR816,,,BG,Y,ORA_BUY
2515,NO-OP,783378839,7833790294,ITEM,FGR809,,,BG,Y,ORA_BUY
2516,NO-OP,783378839,7833790398,ITEM,MVN2749,,,BX,Y,ORA_BUY


In [21]:
PriceListCharges_int=data_joined[['OP_CODE_CREATE','PRICELIST_ID','SOURCE_CHARGE_ID','NEW_SOURCE_CHARGE_ID',
'PARENT_ENTITY_TYPE_CODE','SOURCE_PRICELIST_ITEM_ID','CHARGE_LINE_NUMBER','CHARGE_DEFINITION_CODE','PRICE_PERIODICITY_NAME',
'PRICE_PERIODICITY', 'CALCULATION_METHOD_CODE','B1 EXVAT','CALCULATION_TYPE_CODE', 'COST_CALCULATION_AMOUNT',
 'CALCULATION_AMOUNT','PRICING_BASIS','PRICING_BASIS_ID','CAN_ADJUST_FLAG','START_DATE']]
PriceListCharges_int.head(3)

,OP_CODE_CREATE,PRICELIST_ID,SOURCE_CHARGE_ID,NEW_SOURCE_CHARGE_ID,PARENT_ENTITY_TYPE_CODE,SOURCE_PRICELIST_ITEM_ID,CHARGE_LINE_NUMBER,CHARGE_DEFINITION_CODE,PRICE_PERIODICITY_NAME,PRICE_PERIODICITY,CALCULATION_METHOD_CODE,B1 EXVAT,CALCULATION_TYPE_CODE,COST_CALCULATION_AMOUNT,CALCULATION_AMOUNT,PRICING_BASIS,PRICING_BASIS_ID,CAN_ADJUST_FLAG,START_DATE
0,CREATE,783378830,78337882966,,PRICE_LIST_ITEM,7833788297,2,QP_SALE_PRICE,,,PRICE,32.000000,,,,,,Y,2022-11-07 15:09:48
1,CREATE,783378830,78337882967,,PRICE_LIST_ITEM,7833788298,2,QP_SALE_PRICE,,,PRICE,29.990000,,,,,,Y,2022-11-07 15:09:48
2,CREATE,783378830,78337882968,,PRICE_LIST_ITEM,7833788299,2,QP_SALE_PRICE,,,PRICE,177.316667,,,,,,Y,2022-11-07 15:09:48


## Output CSVs to Downloads Folder

In [25]:
headers_output_file = r'\PriceListsHeaders.csv'
Pricelists_int_unique.to_csv(download_path + headers_output_file,header=False,index=False)
items_output_file = r'\PriceListsItems.csv'
PriceListItems_int.to_csv(download_path + items_output_file,header=False,index=False)
charges_output_file = r'\PriceListsCharges.csv'
PriceListCharges_int.to_csv(download_path + charges_output_file,header=False,index=False)


## Create audit log

In [26]:
auditlog=data_joined[['BATCH_NAME','PRICELIST_ID','Oracle_name',
'SOURCE_PRICELIST_ITEM_ID','NPC','SOURCE_CHARGE_ID','NEW_SOURCE_CHARGE_ID',
'PRICING_UOM_CODE','CHARGE_LINE_NUMBER','B1 EXVAT','START_DATE']]
auditlog.head(3)

,BATCH_NAME,PRICELIST_ID,Oracle_name,SOURCE_PRICELIST_ITEM_ID,NPC,SOURCE_CHARGE_ID,NEW_SOURCE_CHARGE_ID,PRICING_UOM_CODE,CHARGE_LINE_NUMBER,B1 EXVAT,START_DATE
0,LS-2022-11-07-15-09,783378830,National,7833788297,FCB18421,78337882966,,EA,2,32.000000,2022-11-07 15:09:48
1,LS-2022-11-07-15-09,783378830,National,7833788298,FCB18512,78337882967,,EA,2,29.990000,2022-11-07 15:09:48
2,LS-2022-11-07-15-09,783378830,National,7833788299,FAM20380,78337882968,,EA,2,177.316667,2022-11-07 15:09:48


In [27]:
audit_log_filename=r'\MB_reactivate20221104take3.xlsx'
auditlog.to_excel(download_path +audit_log_filename,index=False)